<a href="https://colab.research.google.com/github/Pranesh1306/Pranesh1306/blob/main/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [2]:
def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size, interpolation=cv2.INTER_NEAREST)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    image = image / 255.0  # Normalize by dividing by 255
    return image

def preprocess_label(label_path, target_size):
    label = cv2.imread(label_path)
    label = cv2.resize(label, target_size, interpolation=cv2.INTER_NEAREST)
    label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    label = label / 255.0  # Normalize by dividing by 255
    return label

def preprocess_data(image_folder, label_folder, target_size):
    images = []
    labels = []

    image_files = os.listdir(image_folder)
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, image_file)

        image = preprocess_image(image_path, target_size)
        label = preprocess_image(label_path, target_size)

        images.append(image)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [3]:
# Paths to the image and label folders
import zipfile
zip_file_path_1 = "images.zip"
zip_file_path_2 = "labels.zip"

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path_1, 'r') as zip_ref:
    zip_ref.extractall('images')  # Extract to the 'images' directory
# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path_2, 'r') as zip_ref:
    zip_ref.extractall('labels')  # Extract to the 'images' directory    
    

In [4]:
image_folder = "/content/images"
label_folder = "/content/labels"

In [5]:
# Preprocess the data
target_size = (256, 256)
images, labels = preprocess_data(image_folder,label_folder, target_size)

In [6]:
np.shape(images)

(220, 256, 256, 3)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3
)
# Convert the tuples to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Reshape the images and labels to match the expected input shape
reshaped_images = images.reshape(-1, target_size[0], target_size[1], 3)
reshaped_labels = labels.reshape(-1, target_size[0], target_size[1], 3)

# Generate augmented images
augmented_images = []
augmented_labels = []
for image in reshaped_images:
    augmented_images.extend(datagen.flow(np.expand_dims(image, axis=0), batch_size=1)[0])
for label in reshaped_labels:
    augmented_labels.extend(datagen.flow(np.expand_dims(label, axis=0), batch_size=1)[0])

# Convert the augmented images back to numpy array format
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

In [8]:
# Combine original and augmented images for training
combined_images = np.concatenate((images, augmented_images), axis=0)
combined_labels = np.concatenate((labels, augmented_labels), axis=0)

# Convert the combined dataset back to separate arrays
combined_images = np.array(list(combined_images))
combined_labels = np.array(list(combined_labels))

train_images, test_images, train_labels, test_labels = train_test_split(combined_images,combined_labels, test_size=0.2, random_state=42)
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [9]:
np.shape(train_images)

(352, 256, 256, 3)

In [10]:
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00


In [11]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB0

def unet_model(input_shape):
    inputs = Input(input_shape)

    # Downsample path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Upsample path
    up4 = UpSampling2D(size=(2, 2))(pool2)
    up4 = Conv2D(128, 2, activation='relu', padding='same')(up4)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(merge4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Conv2D(64, 2, activation='relu', padding='same')(up5)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Output
    outputs = Conv2D(3, 1, activation='sigmoid')(conv5)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the U-Net model with EfficientNet backbone
input_shape = (256, 256, 3)  # Adjust if necessary
model = unet_model(input_shape)

# Load pre-trained EfficientNet weights
base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 16  # Adjust if necessary
epochs = 10 # Adjust if necessary
model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_images, test_labels))


16804768/16804768 [==============================] - 0s 0us/step
Epoch 1/10
22/22 [==============================] - 1576s 71s/step - loss: 0.2662 - accuracy: 0.4934 - val_loss: 0.1331 - val_accuracy: 0.9845
Epoch 2/10
22/22 [==============================] - 1556s 71s/step - loss: 0.1273 - accuracy: 0.3841 - val_loss: 0.1124 - val_accuracy: 0.6641
Epoch 3/10
22/22 [==============================] - 1530s 70s/step - loss: 0.1218 - accuracy: 0.4493 - val_loss: 0.1077 - val_accuracy: 0.0825
Epoch 4/10
22/22 [==============================] - 1492s 68s/step - loss: 0.1191 - accuracy: 0.3913 - val_loss: 0.1119 - val_accuracy: 0.9471
Epoch 5/10
22/22 [==============================] - 1494s 68s/step - loss: 0.1173 - accuracy: 0.3816 - val_loss: 0.1043 - val_accuracy: 0.8901
Epoch 6/10
22/22 [==============================] - 1494s 68s/step - loss: 0.1145 - accuracy: 0.2625 - val_loss: 0.1049 - val_accuracy: 0.5846
Epoch 7/10
22/22 [==============================] - 1492s 68s/step - loss: 0.

In [12]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Make predictions on the test set
predictions = model.predict(test_images)

# Calculate metrics
def calculate_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def calculate_specificity(y_true, y_pred):
    true_negative = np.sum((1 - y_true) * (1 - y_pred))
    true_negative_rate = true_negative / np.sum(1 - y_true)
    return true_negative_rate

def calculate_sensitivity(y_true, y_pred):
    true_positive = np.sum(y_true * y_pred)
    true_positive_rate = true_positive / np.sum(y_true)
    return true_positive_rate

accuracy = model.evaluate(test_images, test_labels)[1]
iou = calculate_iou(test_labels, predictions)
specificity = calculate_specificity(test_labels, predictions)
sensitivity = calculate_sensitivity(test_labels, predictions)

print("Accuracy:", accuracy)
print("IOU:", iou)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

3/3 [==============================] - 93s 29s/step - loss: 0.1011 - accuracy: 0.1762
Accuracy: 0.17621733248233795
IOU: 0.034485036676580254
Specificity: 0.9816249883193205
Sensitivity: 0.1515177485692948
